# In this notebook, I am going to run the model by using Linkedin Data

In [1]:
##Import Libraries
import pandas as pd
import requests
import pickle
import re
import string
import unidecode 
import wordninja
import docx2txt
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from autocorrect import Speller 
from nltk.corpus import stopwords 
from nltk import word_tokenize 
import seaborn as sns, numpy as np
import matplotlib.pyplot as plt
from nltk.stem import SnowballStemmer
import time
import random

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from alive_progress import alive_bar
pd.set_option("display.max_rows", None, "display.max_columns", None)

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics.pairwise import cosine_similarity


In [108]:
##pulling data from Linkedin API
##limit:20, used: 4
import requests

url = "https://linkedin-jobs-search.p.rapidapi.com/"

payload = {
	"search_terms": "analyst",
	"location": "berlin",
	"fetch_full_text": "yes"
}
headers = {
	"content-type": "application/json",
	"X-RapidAPI-Host": "linkedin-jobs-search.p.rapidapi.com",
	"X-RapidAPI-Key": "e59a85ffc9mshd439a8fc17a686ep17369fjsn188a0cc3d5dd"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

[{"job_url": "https://de.linkedin.com/jobs/view/quality-analyst-turkish-speaker-at-tiktok-3070525045?refId=T74q54AY%2FiHZFPZpRXCc6w%3D%3D&trackingId=QF5dwJKXCWYKNs8fj6SEDA%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card", "linkedin_job_url_cleaned": "https://de.linkedin.com/jobs/view/quality-analyst-turkish-speaker-at-tiktok-3070525045", "company_name": "TikTok", "company_url": "https://www.linkedin.com/company/tiktok?trk=public_jobs_jserp-result_job-search-card-subtitle", "linkedin_company_url_cleaned": "https://www.linkedin.com/company/tiktok", "job_title": "Quality Analyst-Turkish Speaker", "job_location": "Berlin, Berlin, Germany", "posted_date": "2022-04-19", "normalized_company_name": "Tiktok", "full_text": "\n\n\nResponsibilitiesTikTok is the leading destination for short-form mobile video. Our mission is to inspire creativity and bring joy. Our Trust & Safety team's commitment is to keep our online community safe. We have invested heavily in human and machin

In [109]:
linkedin_data = pd.json_normalize(response.json())

In [110]:
linkedin_data.head()


,job_url,linkedin_job_url_cleaned,company_name,company_url,linkedin_company_url_cleaned,job_title,job_location,posted_date,normalized_company_name,full_text,recruiter_name,linkedin_recruiter_url
0,https://de.linkedin.com/jobs/view/quality-anal...,https://de.linkedin.com/jobs/view/quality-anal...,TikTok,https://www.linkedin.com/company/tiktok?trk=pu...,https://www.linkedin.com/company/tiktok,Quality Analyst-Turkish Speaker,"Berlin, Berlin, Germany",2022-04-19,Tiktok,\n\n\nResponsibilitiesTikTok is the leading de...,,
1,https://de.linkedin.com/jobs/view/analyst-at-e...,https://de.linkedin.com/jobs/view/analyst-at-e...,Engel & Völkers Commercial Berlin,https://de.linkedin.com/company/engelvoelkersc...,https://de.linkedin.com/company/engelvoelkersc...,Analyst,"Berlin, Germany",2022-05-19,Engelvoelkerscommercial,\n\n\n We are hiring! wir suchen dringe...,"Nachricht an Moritz Chrambach, CFA",https://de.linkedin.com/in/mchrambach
2,https://de.linkedin.com/jobs/view/service-oper...,https://de.linkedin.com/jobs/view/service-oper...,Tesla,https://www.linkedin.com/company/tesla-motors?...,https://www.linkedin.com/company/tesla-motors,Service Operations Analyst - Business Performa...,"Berlin, Berlin, Germany",2022-03-24,Tesla motors,"\n\n\nThe RoleAt Tesla, our Service Operations...",,
3,https://de.linkedin.com/jobs/view/financial-an...,https://de.linkedin.com/jobs/view/financial-an...,OneFootball,https://de.linkedin.com/company/onefootball?tr...,https://de.linkedin.com/company/onefootball,Financial Analyst,"Berlin, Berlin, Germany",2022-05-06,Onefootball,\n\n\n Join OneFootball and become a vi...,,
4,https://de.linkedin.com/jobs/view/junior-logis...,https://de.linkedin.com/jobs/view/junior-logis...,Amazon,https://www.linkedin.com/company/amazon?trk=pu...,https://www.linkedin.com/company/amazon,Junior Logistics Analyst (m/f/x) - Barcelona,"Berlin, Berlin, Germany",2022-05-07,Amazon,\n\n\nJob SummaryDESCRIPTIONWe’re expanding ou...,,


In [111]:
pickle.dump(linkedin_data, open("linkedin_data.pkl", "wb"))

In [4]:
linkedin_data = pd.DataFrame()
linkedin_data = pd.read_pickle('linkedin_data.pkl')

In [2]:
##pull the sample resume
resume = docx2txt.process("Sample_CV.docx")


In [3]:
johana = docx2txt.process("Yuan Feng,Yi Bin Johana Apr 2022.docx")

In [5]:
##only extracting the main information I need for the model
data_jobdescription = pd.DataFrame(linkedin_data['full_text'])

In [6]:
data_jobdescription.head()

,full_text
0,\n\n\nResponsibilitiesTikTok is the leading de...
1,\n\n\n We are hiring! wir suchen dringe...
2,"\n\n\nThe RoleAt Tesla, our Service Operations..."
3,\n\n\n Join OneFootball and become a vi...
4,\n\n\nJob SummaryDESCRIPTIONWe’re expanding ou...


In [7]:
## First round cleaning
def clean_text_round1(text):
    ##make all word lower case, remove link, email address
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[\w\.-]+@[\w\.-]+', " ", text)
    #turning all languages to unidecode 
    text = unidecode.unidecode(text)
    ##remove extra white space
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', text)
    ##removing the basic punctuation
    for i in string.punctuation: 
        text = Without_whitespace.replace(i , ' ')
        
    ##removing extra punctuation   
    text = Without_whitespace.replace('?', ' ').replace(')', ' ').replace('(',' ').replace('•', ' • ').replace(":", " ").replace(',',' ').replace('.', ' ').replace('/',' ').replace('!', " ").replace(';', ' ').replace('*', ' ').replace('"', ' ').replace('-',' ').replace('”', " ").replace('>', ' ').replace('-',' ').replace("'s", ' ')
    return text

round1 = lambda x: clean_text_round1(x)

In [8]:
def clean_text_round2(data_column):
    word_count = pd.DataFrame(' '.join(data_column).split()).value_counts()
    list2=[]
    list3 = data_column.values.tolist()
    with alive_bar(force_tty = True) as bar: 
        for a in range(0, len(data_column)): 
            list1 = []
            for b in range(0, len(list3[a].split())):
                if word_count[list3[a].split()[b]] > 4:
                    list1.append(list3[a].split()[b])
                else:
                    text = wordninja.split(list3[a].split()[b])
                    for i in range(0, len(text)):
                        list1.append(text[i])
            list2.append(list1)
        bar()
    for i in range(0, len(data_column)):
        text=' '.join(list2[i])
        data_column.iloc[i] = text
#downside: this code takes too much time to run. 

In [9]:
#replace one digital number, you can consider this step as one step before the cleaning out all the digits. 
def clean_text_round3(text):
    ##replace one digial number which could indicate year of experience into words. 
    text = re.sub(" 1 ", " one ", text)
    text = re.sub(" 2 ", " two ", text)
    text = re.sub(" 3 ", " three ", text)
    text = re.sub(" 4 ", " four ", text)
    text = re.sub(" 5 ", " five ", text)
    text = re.sub(" 6 ", " six ", text)
    text = re.sub(" 7 ", " seven ", text)
    text = re.sub(" 8 ", " eight ", text)
    text = re.sub(" 9 ", " nine ", text)
    text = re.sub(" 10 ", " ten ", text)
    text = re.sub(" 1+ ", " one ", text)
    text = re.sub(" 2+ ", " two ", text)
    text = re.sub(" 3+ ", " three ", text)
    text = re.sub(" 4+ ", " four ", text)
    text = re.sub(" 5+ ", " five ", text)
    text = re.sub(" 6+ ", " six ", text)
    text = re.sub(" 7+ ", " seven ", text)
    text = re.sub(" 8+ ", " eight ", text)
    text = re.sub(" 9+ ", " nine ", text)
    text = re.sub(" 10+ ", " ten ", text)
    return text

round3 = lambda x: clean_text_round3(x)


In [10]:
#@cleaning all digits in the dateset, aiming for telephone number or zip code or job ID etc. 
def clean_text_round4(text):
    ##removing all digital 
    text = re.sub('\d', '', text)
    return text

round4 = lambda x: clean_text_round4(x)

In [11]:
data_clean = pd.DataFrame(data_jobdescription.full_text.apply(round1))

In [12]:
clean_text_round2(data_clean['full_text'])

|████████████████████████████████████████| (!) 1 in 2.3s (0.44/s) 


In [13]:
data_clean = pd.DataFrame(data_jobdescription.full_text.apply(round3))
data_clean = pd.DataFrame(data_jobdescription.full_text.apply(round4))

In [14]:
d = {'resume': [resume]}
df_resume = pd.DataFrame(data = d)

In [16]:
d_jo = {'resume': [johana]}
df_resume_jo = pd.DataFrame(data = d_jo)

In [15]:
df_resume = pd.DataFrame(df_resume['resume'].apply(round1))
clean_text_round2(df_resume['resume'])
df_resume = pd.DataFrame(df_resume['resume'].apply(round3))
df_resume = pd.DataFrame(df_resume['resume'].apply(round4))
df_resume


|████████████████████████████████████████| (!) 1 in 0.2s (5.67/s) 


,resume
0,maria lisa email phone five five career summa...


In [17]:
df_resume_jo = pd.DataFrame(df_resume_jo['resume'].apply(round1))
clean_text_round2(df_resume_jo['resume'])
df_resume_jo = pd.DataFrame(df_resume_jo['resume'].apply(round3))
df_resume_jo = pd.DataFrame(df_resume_jo['resume'].apply(round4))
df_resume_jo

|████████████████████████████████████████| (!) 1 in 0.3s (3.92/s) 


,resume
0,yi bin johan a yuan feng email mobile educat...


In [18]:
## comparing the resume to every job description in the dataset
similarity_scores = []
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
for i in range(0,len(data_clean)):
    job_description = data_clean.iloc[i]['full_text']
    text = [johana, job_description]
    count_matrix = cv.fit_transform(text)
    similarity_scores.append(cosine_similarity(count_matrix)[0][1])

In [25]:
## comparing the resume to every job description in the dataset
similarity_scores = []
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
for i in range(0,len(data_clean)):
    job_description = data_clean.iloc[i]['full_text']
    text = [resume, job_description]
    count_matrix = cv.fit_transform(text)
    similarity_scores.append(cosine_similarity(count_matrix)[0][1])

In [26]:

job_score = pd.DataFrame(similarity_scores, columns = ['similarity_scores'])


In [27]:
linkedin_company_url_cleaned = []
jobdescription = []
job_scores = []
for i in job_score['similarity_scores'].nlargest(n=10).index.to_list():
    linkedin_company_url_cleaned.append(linkedin_data.iloc[i]['linkedin_company_url_cleaned'])
    jobdescription.append(linkedin_data.iloc[i]['full_text'])
    job_scores.append(job_score.iloc[i]['similarity_scores'])

In [28]:
top_10_jobs_list = list(zip(linkedin_company_url_cleaned,jobdescription,job_scores))

In [29]:
top_10_jobs = pd.DataFrame(top_10_jobs_list, columns = ['advertiserurl','jobdescription','job_scores'])

In [30]:
top_10_jobs

,advertiserurl,jobdescription,job_scores
0,https://de.linkedin.com/company/baobabinsurance,\n\n\nYour missionIf you have a gift of innova...,0.263844
1,https://de.linkedin.com/company/zalando,\n\n\nTHE ROLE & THE TEAMAt zStudios Content A...,0.184889
2,https://de.linkedin.com/company/babbel-com,\n\n\n Babbel is driven by a purpose: C...,0.176248
3,https://de.linkedin.com/company/goflink,\n\n\nCompany DescriptionWe are Flink - your o...,0.154661
4,https://de.linkedin.com/company/y42-business-i...,"\n\n\n Y42 is a fast-growing company, f...",0.135852
5,https://de.linkedin.com/company/dimpact-io,"\n\n\n🚀The Mission:Our client, a Stealth Start...",0.117884
6,https://de.linkedin.com/company/vaayu,"\n\n\n At Vaayu, we are on a mission to...",0.109274
7,https://de.linkedin.com/company/basf,"\n\n\n Along the order-to-cash process,...",0.108700
8,https://de.linkedin.com/company/choco-app,"\n\n\n At Choco, we’re striving to buil...",0.106332
9,https://de.linkedin.com/company/zolar-gmbh,\n\n\nYour role at zolarYou continuously impro...,0.101727


i=[]

for a in range(0,2):
    b = str(a)
    c = "".join(('"', b, '"'))
        
    url = "https://indeed11.p.rapidapi.com/"

    payload = {
        "search_terms": "startup",
        "location": "Berlin",
        "page": c,
        "fetch_full_text": "yes"
        }
    headers = {
            "content-type": "application/json",
            "X-RapidAPI-Host": "indeed11.p.rapidapi.com",
            "X-RapidAPI-Key": "e59a85ffc9mshd439a8fc17a686ep17369fjsn188a0cc3d5dd"
        }

    response_indeed = requests.request("POST", url, json=payload, headers=headers)
    j = response_indeed.json()
    i.append(j)
    print(response_indeed.text)
    print(j)
    print(i)
    d= random.randint(1, 2)
    time.sleep(d)